# <u>Deepfake Video Detection Project</u>

In [6]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2 as cv
import glob

## MetaData of the DataSet


In [7]:
train_sample_metadata = pd.read_json('/content/drive/MyDrive/Dataset/train_sample_videos/metadata.json').T
train_sample_metadata.head()

FileNotFoundError: File /content/drive/MyDrive/Dataset/train_sample_videos/metadata.json does not exist

- Index is the Name of the Video
- Label represents the type of video it is.
 - if the label is Fake orginal column contains its orginal video name else it is None


In [ ]:
train_sample_metadata.info()

In [ ]:
train_sample_metadata['label'].value_counts()

In [ ]:
train_sample_metadata.groupby('label')['label'].count().plot(figsize=(10, 5), kind='bar', title='Distribution of Labels in the Training Set')
plt.show()

Unique Values

In [ ]:
def unique_values(data):
    total = data.count()
    tt = pd.DataFrame(total)
    tt.columns = ['Total']
    uniques = []
    for col in data.columns:
        unique = data[col].nunique()
        uniques.append(unique)
    tt['Uniques'] = uniques
    return(np.transpose(tt))

In [ ]:
unique_values(train_sample_metadata)

- Uniquess of label =2 tells that there are 2 unique values("FAKE", "REAL" )
- Uniquess of original =209 tells that there are 209 unique video names in the train_sample_metadata dataframe

### Most Frequent items

In [ ]:
def most_frequent_values(data):
    total = data.count()
    tt = pd.DataFrame(total)
    tt.columns = ['Total']
    items = []
    vals = []
    for col in data.columns:
        itm = data[col].value_counts().index[0]
        val = data[col].value_counts().values[0]
        items.append(itm)
        vals.append(val)
    tt['Most frequent item'] = items
    tt['Frequency'] = vals
    tt['Percent from total'] = np.round(vals / total * 100, 3)
    return(np.transpose(tt))


In [ ]:
most_frequent_values(train_sample_metadata)

- FAKE label is the most frequently occuring item in label column with percentage of 80.75%
- which indicates REAL label is occuring with percentage of 19.25%
- atvmxvwyns.mp4 is the most frequently occuring video

In [ ]:
original_counts = pd.DataFrame(train_sample_metadata['original'].value_counts())
original_counts.head(10)

### some of the Fake Video names

In [ ]:
fake_train_sample_video = list(train_sample_metadata.loc[train_sample_metadata.label=='FAKE'].sample(10).index)
fake_train_sample_video

### some of the REAL Video names

In [8]:
real_train_sample_video = list(train_sample_metadata.loc[train_sample_metadata.label=='REAL'].sample(10).index)    # returning the index value which is video name
real_train_sample_video

NameError: name 'train_sample_metadata' is not defined

### Function to display the image from the video

In [ ]:
def display_image_from_video(video_path):
    '''
    input: video_path - path for video
    process:
    1. perform a video capture from the video
    2. read the image
    3. display the image
    '''
    capture_image = cv.VideoCapture(video_path)
    ret, frame = capture_image.read()
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)   # converting the frame color to RGB
    ax.imshow(frame)


## Fake Video Images

In [ ]:
for video in fake_train_sample_video:
  display_image_from_video(os.path.join("/content/drive/MyDrive/Dataset/train_sample_videos/"+video))
  print()
  print()


## REAL Video Images

In [ ]:
for video in real_train_sample_video:
  display_image_from_video(os.path.join("/content/drive/MyDrive/Dataset/train_sample_videos/"+video))
  print()
  print()

### Function to play the video

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def play_video(video_file):
    '''
    Display video
    param: video_file - the name of the video file to display
    param: subset - the folder where the video file is located (can be TRAIN_SAMPLE_FOLDER or TEST_Folder)
    '''
    video_url = open(video_file,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(video_url).decode()
    return HTML("""<video width=500 controls><source src="%s" type="video/mp4"></video>""" % data_url)

In [ ]:
play_video("/content/drive/MyDrive/Dataset/train_sample_videos/aapnvogymq.mp4")

### Video in a Darkness

In [9]:
play_video("/content/drive/MyDrive/Dataset/train_sample_videos/atvmxvwyns.mp4")

NameError: name 'play_video' is not defined

## Finding Average Frame rate of Videos

In [ ]:
videos = glob.glob('/content/drive/MyDrive/Dataset/train_sample_videos/*.mp4')
frame_cnt = []
for video in videos:
  capture = cv.VideoCapture(video)
  frame_cnt.append(int(capture.get(cv.CAP_PROP_FRAME_COUNT)))
print("Frames: ",frame_cnt)
print("Avg Frame per video: ",np.mean(frame_cnt))

# Locating a Face within an Image

In [ ]:
!pip install face_recognition

In [ ]:
def image_from_video(video_path):
    '''
    input: video_path - path for video
    process:
    1. perform a video capture from the video
    2. read the image
    3. display the image
    '''
    capture_image = cv.VideoCapture(video_path)
    ret, frame = capture_image.read()
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)   # converting the frame color to RGB
    ax.imshow(frame)

    return frame

In [ ]:
image = image_from_video("/content/drive/MyDrive/Dataset/train_sample_videos/aagfhgtpmv.mp4")

In [ ]:
import face_recognition

In [ ]:
face_locations = face_recognition.face_locations(image)

from PIL import Image


for face_location in face_locations:

  top,right,bottom,left = face_location
  print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

  face_image = image[top:bottom, left:right]
  fig, ax = plt.subplots(1,1, figsize=(5, 5))
  plt.grid(False)
  ax.xaxis.set_visible(False)
  ax.yaxis.set_visible(False)
  ax.imshow(face_image)
